In [3]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [6]:
filename = "data/trumptextmerged2.txt"
raw_text = open(filename, 'r', encoding='utf-8').read()
raw_text = raw_text.lower()

In [7]:
small_subset = raw_text[0:700000]

In [8]:
small_subset

" thank you very much, everybody. thank you. thank you. great honor. please, thank you. sit down, please. its great to be with you. great company and wonderful location. quick flight. its in and out, but we have some big things to say. carol, i want to thank you also for doing such an incredible job at this company. i look at your numbers and im very jealous. a lot of people are very jealous. but im delighted to be back in atlanta, georgia, special place. the hometown of one of the most amazing companies on earth, ups. they never fail. the dedicated men and women of ups are an inspiration to us all. in the face of every challenge, you always come through for your fellow citizens. would you say thats correct? i say its correct.    let me start by expressing my gratitude to every driver, worker, and employee who has contributed to this great success and continue to deliver for america throughout our battle against the china virus, it goes by many different names, about 21 that i can figu

In [9]:
# create mapping of unique chars to integers
chars = sorted(list(set(small_subset)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

In [10]:
n_chars = len(small_subset)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)

Total Characters:  700000
Total Vocab:  53


In [11]:
chars

[' ',
 '!',
 '$',
 '%',
 "'",
 '(',
 ')',
 '+',
 ',',
 '-',
 '.',
 '/',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 ';',
 '?',
 '[',
 ']',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z']

In [12]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 120
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
	seq_in = small_subset[i:i + seq_length]
	seq_out = small_subset[i + seq_length]
	dataX.append([char_to_int[char] for char in seq_in])
	dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)

Total Patterns:  699880


In [13]:
# reshape X to be [samples, time steps, features]
X = np.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

In [14]:
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [17]:
# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [33]:
model.fit(X, y, epochs=12, batch_size=128, callbacks=callbacks_list)

Epoch 1/12
74880/74880 [==============================] - 657s 9ms/step - loss: 2.9554

Epoch 00001: loss improved from inf to 2.95544, saving model to weights-improvement-01-2.9554.hdf5
Epoch 2/12
74880/74880 [==============================] - 654s 9ms/step - loss: 2.7939

Epoch 00002: loss improved from 2.95544 to 2.79385, saving model to weights-improvement-02-2.7939.hdf5
Epoch 3/12
74880/74880 [==============================] - 661s 9ms/step - loss: 2.7244

Epoch 00003: loss improved from 2.79385 to 2.72441, saving model to weights-improvement-03-2.7244.hdf5
Epoch 4/12
74880/74880 [==============================] - 716s 10ms/step - loss: 2.6745

Epoch 00004: loss improved from 2.72441 to 2.67449, saving model to weights-improvement-04-2.6745.hdf5
Epoch 5/12
74880/74880 [==============================] - 748s 10ms/step - loss: 2.6324

Epoch 00005: loss improved from 2.67449 to 2.63244, saving model to weights-improvement-05-2.6324.hdf5
Epoch 6/12
74880/74880 [=======================

In [18]:
import sys

In [23]:
# load the network weights
filename = "weights-improvement-02-2.4054.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [24]:
int_to_char = dict((i, c) for i, c in enumerate(chars))

In [27]:
# pick a random seed
start = np.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print("Seed:")
print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
# generate characters
for i in range(100):
	x = np.reshape(pattern, (1, len(pattern), 1))
	x = x / float(n_vocab)
	prediction = model.predict(x, verbose=0)
	index = np.argmax(prediction)
	result = int_to_char[index]
	seq_in = [int_to_char[value] for value in pattern]
	sys.stdout.write(result)
	pattern.append(index)
	pattern = pattern[1:len(pattern)]
print("\nDone.")

Seed:
" vestment at its orion assembly plant right here in michigan. come on, general motors, lets go. general motors gave us a  "
lot of the forernon  00 bnl the warl torld teat werl torking the world  bnd the werl fave teat werl 
Done.
